In [2]:
path1 = './data/salmonella_spi1_region.fna'

In [101]:
def read_fasta(path):
    with open(path,'r') as f:
        f_line = f.readlines()
        descriptor_string = f_line[0].rstrip()
        seq=f_line[1:]
        sequence = ''
        for i,str in enumerate(seq):
            seq[i] = seq[i].removesuffix('\n')
            sequence += seq[i]
    return (descriptor_string, sequence)

In [102]:
path2 = './data/Lambda_phage.fasta'
rest_enzymes = {'HindIII':'AAGCTT','EcoRI':'GAATTC','KpnI':'GGTACC'}

In [103]:
rest_enzymes

{'HindIII': 'AAGCTT', 'EcoRI': 'GAATTC', 'KpnI': 'GGTACC'}

In [104]:
def restriction_sites(seq, recog_seq):
    list_m=[]
    for i,_ in enumerate(seq):
        if seq[i:i+len(recog_seq)]==recog_seq:
            list_m.append(i+1)
    return list_m

In [105]:
desc, sequence = read_fasta(path2)
print(desc)
list_1 = []
for key, values in rest_enzymes.items():
    m = (key,restriction_sites(sequence,values))
    list_1.append(m)

print(list_1)

>Lambda_NEB
[('HindIII', [23130, 25157, 27479, 36895, 37459, 37584, 44141]), ('EcoRI', [21226, 26104, 31747, 39168, 44972]), ('KpnI', [17053, 18556])]


In [106]:
def gc_blocks(seq, block_size):
    list1=[]
    for m in range(0,len(seq),block_size):
        if len(seq)-m>block_size:
            list1.append(seq[m:m+block_size])
    list2=[]
    for i, str in enumerate(list1):
        list2.append((str.count('G')+str.count('C'))/block_size)
    return tuple(list1),tuple(list2)

In [107]:
def gc_map(seq,block_size,gc_thresh):
    string_out=''
    blocks,gc = gc_blocks(seq,block_size)
    for i, str in enumerate(blocks):
        if gc[i]<gc_thresh:
            string_out+=str.lower()
        else:
            string_out+=str.upper()
    return string_out

gc_map('ATGACTACGT', 4, 0.4)

'atgaCTAC'

In [108]:
import os
desc_sal,seq_sal = read_fasta(path1)
string = gc_map(seq_sal,1000,0.45)
fname = 'GC_mapped_sequence.txt'
if os.path.isfile(fname):
    raise RuntimeError('File already exists')

with open(fname, 'w') as f:
    for j in range(0,len(string)//60 +1):
        f.write(string[j*60:(j+1)*60])
        f.write('\n')

In [119]:
#Does_not_work
seq='GATGAATGATGTCCCGGGGTAGAAAACGTTAG'
def longest_orf(seq):
    start_sites = restriction_sites(seq,'ATG')
    end_sites = restriction_sites(seq,'TGA')+restriction_sites(seq,'TAG')+restriction_sites(seq,'TAA')
    listm=[]
    #print(start_sites,end_sites)
    for i in range(0,len(start_sites)):
        for j in range(0,len(end_sites)):
            #print(end_sites[-1-j]-start_sites[i])+3
            if (end_sites[j]-start_sites[i])%3 ==0:
                l=(start_sites[i],end_sites[j],end_sites[j]-start_sites[i])
                listm.append(l)
    print(listm)
m = longest_orf(seq)
print(m)

[(2, 20, 18), (6, 3, -3), (6, 30, 24), (9, 3, -6), (9, 30, 21)]
None


In [120]:
orf_sal = longest_orf(seq_sal)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [121]:
import bootcamp_utils

def dna_to_prot(seq):
    seq_tr=''
    for i in range(0,len(seq),3):
        if seq[i:i+3] in bootcamp_utils.codons:
            seq_tr+=bootcamp_utils.codons[seq[i:i+3]]
    return seq_tr

In [122]:
print(dna_to_prot(orf_sal))

TypeError: object of type 'NoneType' has no len()